# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [83]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [84]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fale old settlement,-9.3852,-171.2468,28.09,75,8,5.22,TK,1724807787
1,1,sao felix do xingu,-6.6447,-51.9950,29.79,30,100,0.66,BR,1724807788
2,2,ancud,-41.8697,-73.8203,5.03,88,70,2.41,CL,1724807791
3,3,grytviken,-54.2811,-36.5092,3.20,84,100,3.70,GS,1724807792
4,4,puerto deseado,-47.7503,-65.8938,6.11,65,98,7.99,AR,1724807793


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [85]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humdidity",
    color = "Red"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [86]:
# Narrow down cities that fit criteria and drop any results with null values
my_ideal_cities = city_data_df.loc[(city_data_df['Max Temp']<20) & (city_data_df['Wind Speed']<4.5) & (city_data_df['Cloudiness']>10), :]

# Drop any rows with null values
#my_ideal_cities = my_ideal_cities.dropna()
# Display sample data
my_ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,ancud,-41.8697,-73.8203,5.03,88,70,2.41,CL,1724807791
3,3,grytviken,-54.2811,-36.5092,3.20,84,100,3.70,GS,1724807792
10,10,udachny,66.4167,112.4000,11.37,66,88,2.71,RU,1724807801
11,11,qaqortoq,60.7167,-46.0333,5.25,84,100,2.27,GL,1724807802
15,15,ushuaia,-54.8000,-68.3000,5.81,65,75,3.60,AR,1724807579
...,...,...,...,...,...,...,...,...,...,...
215,215,chibougamau,49.9168,-74.3659,13.79,72,100,4.12,CA,1724808063
221,221,ola,59.5833,151.2833,13.05,82,60,1.92,RU,1724808071
223,223,mugia,43.1041,-9.2179,18.28,90,82,1.39,ES,1724808073
225,225,lesogorsk,56.0458,99.5136,14.14,76,16,3.76,RU,1724808076


### Step 3: Create a new DataFrame called `hotel_df`.

In [87]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,ancud,CL,-41.8697,-73.8203,88,
3,grytviken,GS,-54.2811,-36.5092,84,
10,udachny,RU,66.4167,112.4000,66,
11,qaqortoq,GL,60.7167,-46.0333,84,
15,ushuaia,AR,-54.8000,-68.3000,65,
...,...,...,...,...,...,...
215,chibougamau,CA,49.9168,-74.3659,72,
221,ola,RU,59.5833,151.2833,82,
223,mugia,ES,43.1041,-9.2179,90,
225,lesogorsk,RU,56.0458,99.5136,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [88]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
"categories":categories,
"limit":1,
"apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    my_row_lat = row["Lat"]
    my_row_lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    filters = f"circle:{my_row_lon},{my_row_lat},{radius}"
    bias = f"proximity:{my_row_lon},{my_row_lat}"
    params["filter"] = filters
    params["bias"] = bias


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ancud - nearest hotel: Cabañas
grytviken - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
qaqortoq - nearest hotel: Hotel Qaqortoq
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
puerto natales - nearest hotel: Dorotea Patagonia Hostel
yellowknife - nearest hotel: No hotel found
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
tazovsky - nearest hotel: Тазовчанка
thompson - nearest hotel: Thompson Inn
bozkurt - nearest hotel: Abana Kutay Hotel
aasiaat - nearest hotel: SØMA
kogalym - nearest hotel: Сибирь
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
zeya - nearest hotel: Серебряный створ
stornoway - nearest hotel: County Hotel
olonkinbyen - nearest hotel: No hotel found
chonchi - nearest hotel: Hotel Huildín
asau - nearest hotel: Hotel Romanița
blackmans bay - nearest hotel: Villa Howden
antofagasta - nearest hotel: Hotel Colón
san jose de jachal - near

,City,Country,Lat,Lng,Humidity,Hotel Name
2,ancud,CL,-41.8697,-73.8203,88,Cabañas
3,grytviken,GS,-54.2811,-36.5092,84,No hotel found
10,udachny,RU,66.4167,112.4000,66,Вилюй
11,qaqortoq,GL,60.7167,-46.0333,84,Hotel Qaqortoq
15,ushuaia,AR,-54.8000,-68.3000,65,Apart Hotel Aires del Beagle
...,...,...,...,...,...,...
215,chibougamau,CA,49.9168,-74.3659,72,No hotel found
221,ola,RU,59.5833,151.2833,82,No hotel found
223,mugia,ES,43.1041,-9.2179,90,Solpor do Coído
225,lesogorsk,RU,56.0458,99.5136,76,Теремок


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [91]:
#%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
"Lng",
"Lat",
geo = True,
tiles = "EsriNatGeo",
frame_width = 700,
frame_height = 500,
size = "Humdidity",
color = "red",
hover_cols = ["Hotel Name", "Country"],
hover_name = "City"
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)